In [1]:
import pandas as pd 
import numpy as np

## Initial EDA

    shortcut is to use pandas_profiling, sweetviz, and/or autoviz packages.

pandas_profiling (ydata_profiling)

In [12]:
#import ydata_profiling

#profile = ydata_profiling.ProfileReport(df)

#profile

sweetviz

In [7]:
#import sweetviz as sv

#report = sv.analyze(df)

#report.show_notebook()

autoviz (CSV, txt, or JSON formats)

In [10]:
#from autoviz.AutoViz_Class import AutoViz_Class

#AV = AutoViz_Class()

# dft = AV.AutoViz(filename)

## manual data characteristics function

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

def get_df_characteristics(df):

    # dimensions, col names, presence of NaNs, data types
    print(df.info())

    # Missing values
    missing_values = df.isnull().sum().reset_index()
    missing_values.columns = ['Column', 'Missing Values']

    # Summary statistics
    summary_stats = df.describe().reset_index()
    summary_stats.columns = ['Statistic', 'Column'] + summary_stats.columns[2:].tolist()

    # Unique values count for each column
    unique_values = df.nunique().reset_index()
    unique_values.columns = ['Column', 'No. Unique Values']

    # Print the characteristics using tabulate
    print()
    print("Missing values:")
    print(tabulate(missing_values, headers='keys', tablefmt='grid', showindex=False))
    print()
    print("Summary statistics:")
    print(tabulate(summary_stats, headers='keys', tablefmt='grid', showindex=False))
    print()
    print("No. unique values:")
    print(tabulate(unique_values, headers='keys', tablefmt='grid', showindex=False))


## plot feature distributions

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def plot_feature_distributions(df):
    num_plots = len(df.columns)
    num_rows = 2
    num_cols = num_plots // num_rows + num_plots % num_rows

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 10))
    fig.tight_layout(pad=4.0)

    for i, column in enumerate(df.columns):
        ax = axes[i // num_cols, i % num_cols]
        if df[column].dtype.kind in 'biufc': #  'b' for boolean, 'i' for signed integer, 'u' for unsigned integer, 'f' for floating-point, and 'c' for complex.
            sns.histplot(data=df, x=column, ax=ax) # if numeric
        else:
            sns.countplot(data=df, x=column, ax=ax) # if non-numeric

    plt.show()


## plot correlations

    Pearson Correlation: Default. Measures the linear relationship between two variables and is appropriate for continuous variables. Specify by setting correlation_type='pearson'.

    Spearman Correlation: Measures the monotonic relationship between two variables, which captures both linear and non-linear associations. Suitable for continuous or ordinal variables. Specify by setting correlation_type='spearman'.

    Kendall Correlation: Measures the strength of the ordinal association between two variables. Useful for assessing the relationship between non-normally distributed ordinal variables. Specify by setting correlation_type='kendall'.

    Point Biserial Correlation: Measures the association between a continuous variable and a binary variable. Special case of the Pearson correlation. Specify by setting correlation_type='pointbiserial'.

    Cramer's V: Measures the association between two categorical variables and is suitable for nominal variables. Based on chi-square statistics. Specify setting correlation_type='cramers'.

    Phi Coefficient: Measures the association between two binary variables. Similar to point biserial correlation. You can specify it by setting correlation_type='phi'.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def plot_correlation(df, correlation_type='pearson'):
    # Compute correlation matrix
    corr_matrix = df.corr(method=correlation_type)

    # Set up the figure and axes
    fig, ax = plt.subplots(figsize=(20, 16))

    # Create a mask for the upper triangle
    mask = ~corr_matrix.isnull()

    # Plot the correlation matrix heatmap
    sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f", cmap="coolwarm", cbar=True, square=True, ax=ax)

    # Customize the plot
    ax.set_title(f"{correlation_type.capitalize()} Correlation Matrix")
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()

    # Show the plot
    plt.show()


## fill NaN values

In [2]:
def fill_nan(df, method, columns=None):
    if columns is None:
        columns = df.columns  # If columns are not specified, fill NaN values for all columns
    
    if method == "mean":
        filled_df = df.copy()
        filled_df[columns] = filled_df[columns].fillna(filled_df[columns].mean())
    elif method == "median":
        filled_df = df.copy()
        filled_df[columns] = filled_df[columns].fillna(filled_df[columns].median())
    elif method == "mode":
        filled_df = df.copy()
        for column in columns:
            filled_df[column] = filled_df[column].fillna(filled_df[column].mode().iloc[0])
    elif method == "zero":
        filled_df = df.copy()
        filled_df[columns] = filled_df[columns].fillna(0)
    else:
        raise ValueError("Invalid imputation method. Please choose 'mean', 'median', 'mode', or 'zero'.")
    
    return filled_df


## X, y, train test split


- Specify dataframe, and target column as predictor

In [3]:
from sklearn.model_selection import train_test_split

def process_df(df, target_column, test_size, random_state, stratify_var):
    # Splitting into features and target variable
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    
    # Perform train-test split with specified parameters
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=stratify_var)
    
    return X, y, X_train, y_train, X_test, y_test


## One hot encoding entire df

In [4]:
from sklearn.preprocessing import OneHotEncoder

def perform_OHE_df(df):
    # Identify categorical columns
    categorical_cols = df.select_dtypes(include='object').columns.tolist()
    
    # Perform one-hot encoding
    ohe = OneHotEncoder(sparse=False, drop='first')
    transformed = ohe.fit_transform(df[categorical_cols])
    
    # Create new DataFrame with transformed features
    transformed_df = pd.DataFrame(transformed, columns=ohe.get_feature_names_out(categorical_cols))
    
    # Combine transformed features with non-categorical columns
    df_transformed = pd.concat([df.drop(categorical_cols, axis=1), transformed_df], axis=1)
    
    return df_transformed


# example usage
# df_transformed = perform_OHE_df(df)

## One hot encoding specific features

In [5]:
def perform_OHE_feature(df, features):
    # Perform one-hot encoding
    ohe = OneHotEncoder(sparse=False, drop='first')
    transformed = ohe.fit_transform(df[features])
    
    # Create new DataFrame with transformed features
    transformed_df = pd.DataFrame(transformed, columns=ohe.get_feature_names_out(features))
    
    # Drop original columns and combine transformed features with the DataFrame
    df_transformed = df.drop(features, axis=1)
    df_transformed = pd.concat([df_transformed, transformed_df], axis=1)
    
    return df_transformed

# Example usage

# features = ['column1', 'column2'] # specify columns to OHE
# df_transformed = perform_OHE_feature(df, features)

## Ordinal encoding

In [6]:
from sklearn.preprocessing import OrdinalEncoder

def ordinal_encode(df, columns):
    encoder = OrdinalEncoder()
    df[columns] = encoder.fit_transform(df[columns])
    return df

# example usage
# columns_to_encode = ['column1', 'column2']  # Specify the categorical columns to encode
# df_encoded = ordinal_encode(df, columns_to_encode)

## label encoding

In [7]:
from sklearn.preprocessing import LabelEncoder

def label_encode(df, columns):
    encoder = LabelEncoder()
    df[columns] = df[columns].apply(encoder.fit_transform)
    return df


# example usage
#columns_to_encode = ['column1', 'column2']  # Specify the categorical columns to encode
#df_encoded = label_encode(df, columns_to_encode)

## standardisation/normalisation/feature scaling

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def scale_features(df, numerical_features, scaling_method):
    if scaling_method == "standardisation":
        scaler = StandardScaler()
        df[numerical_features] = scaler.fit_transform(df[numerical_features])
    elif scaling_method == "normalisation":
        scaler = MinMaxScaler()
        df[numerical_features] = scaler.fit_transform(df[numerical_features])
    elif scaling_method == "-1":
        # Specify your custom range for scaling
        custom_min = -1
        custom_max = 1
        scaled_range = custom_max - custom_min

        min_val = df[numerical_features].min()
        max_val = df[numerical_features].max()
        df[numerical_features] = custom_min + ((df[numerical_features] - min_val) / (max_val - min_val)) * scaled_range
    else:
        raise ValueError("Invalid scaling method. Please choose 'standardisation', 'normalisation', or '-1'.")

    return df

# example usage
# numerical_features = ['feature1', 'feature2', 'feature3']  # Specify the numerical features to scale
# scaling_method = "standardisation"  # Specify the scaling method ("standardisation", "normalisation", or "-1")

#df_scaled = scale_features(df, numerical_features, scaling_method)


## detect outliers

In [9]:
import numpy as np

def detect_outliers(df, numerical_features, threshold=3):
    z_scores = np.abs((df[numerical_features] - df[numerical_features].mean()) / df[numerical_features].std())
    outlier_mask = z_scores > threshold
    return outlier_mask.any(axis=1)

# example usage
# numerical_features = ['feature1', 'feature2', 'feature3']  # Specify the numerical features for outlier detection
# outlier_threshold = 3  # Specify the threshold for the Z-score method

## handle outliers



In [ ]:
def handle_outliers(df, outlier_mask, strategy="remove", value=None):
    if strategy == "remove":
        df_cleaned = df[~outlier_mask]
    elif strategy == "impute" and value is not None:
        df_cleaned = df.copy()
        df_cleaned.loc[outlier_mask] = value
    else:
        raise ValueError("Invalid outlier handling strategy or missing value for imputation.")
    
    return df_cleaned

# example usage
# outlier_mask = detect_outliers(df, numerical_features, threshold=outlier_threshold)
#
## Remove outliers
# df_cleaned = handle_outliers(df, outlier_mask, strategy="remove")
#
## Impute outliers with a specific value
# df_imputed = handle_outliers(df, outlier_mask, strategy="impute", value=999)

## Feature Selection (RFE)

In [14]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

def perform_recursive_feature_elimination(X, y, n_features):
    estimator = LogisticRegression(solver='liblinear')
    selector = RFE(estimator, n_features_to_select=n_features)
    X_new = selector.fit_transform(X, y)
    selected_features = X.columns[selector.support_]
    return X_new, selected_features


# example usage

#X = ...  # Your feature matrix
#y = ...  # Your target variable

#n_features = 5  # Number of features to select
#X_new, selected_features = perform_recursive_feature_elimination(X, y, n_features)
#print("Selected Features:", selected_features)

## Visualisation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_histogram(df, feature):
    plt.figure(figsize=(8, 6))
    sns.histplot(df[feature].dropna(), kde=True)
    plt.title(f"Histogram of {feature}")
    plt.xlabel(feature)
    plt.ylabel("Frequency")
    plt.show()

def plot_correlation_matrix(df):
    corr_matrix = df.corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
    plt.title("Correlation Matrix")
    plt.show()
